# Preliminar Analysis

In [5]:
import pandas as pd
from google.cloud import bigquery
from tqdm import tqdm
import glob
import pickle

PROJECT_ID = 'siris-datasets'
DATASET_ID = 'openalex'

client = bigquery.Client(project = PROJECT_ID)

/home/siris/2025MESRE/2025MESRE_env/lib/python3.10/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
sql = f"""
WITH french_works AS (
    SELECT DISTINCT wa.WORK_ID
    FROM `{PROJECT_ID}.{DATASET_ID}.works_authorships` wa
    JOIN `{PROJECT_ID}.{DATASET_ID}.institutions` inst
      ON inst.ID = wa.INSTITUTION_ID
    WHERE inst.COUNTRY_CODE = 'FR'
)
SELECT 
    ww.DOI,
    ww.LANGUAGE,
    a.display_name,
    wa.is_corresponding,
    wins.ID AS institution_id,
    wins.COUNTRY_CODE
FROM `{PROJECT_ID}.{DATASET_ID}.works` ww
JOIN french_works fw
    ON ww.ID = fw.WORK_ID
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.works_authorships` wa
    ON wa.WORK_ID = ww.ID
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.institutions` wins
    ON wins.ID = wa.INSTITUTION_ID
LEFT JOIN `{PROJECT_ID}.{DATASET_ID}.authors` a
    ON a.ID = wa.author_id
"""

temp_table = f"{PROJECT_ID}.Adria.temp_results"
job_config = bigquery.QueryJobConfig(
    destination=temp_table,
    write_disposition="WRITE_TRUNCATE"  # overwrite if exists
)

query_job = client.query(sql, job_config=job_config)
query_job.result()  # Wait for the query to finish

# Step 2: Pull results in chunks into Pandas to save memory
row_iterator = client.list_rows(temp_table, page_size=1000000)
for id_x, chunk in enumerate(tqdm(row_iterator.to_dataframe_iterable(), desc="Loading chunks")):
    chunk.dropna(subset = ['DOI', 'display_name']).drop_duplicates(['DOI', 'display_name', 'institution_id']).\
    reset_index(drop = True).to_csv(f'../data/interim/FranceInitial/{id_x}.csv', index = False)

Loading chunks: 332it [45:56,  8.30s/it]


1. publications in French language \
(1.a) how many have more than 50% of the authors with no French affiliations at all? \
(1.b) how many have more than 50% of the authors with no country information?

In [4]:
df_results_france = []
for file in tqdm(glob.glob('../data/interim/FranceInitial/*.csv'), desc="Loading chunks"):
    df_tmp = pd.read_csv(file)
    df_results_france.append(list(df_tmp.DOI.unique()))
len(set([doi for sublist in df_results_france for doi in sublist]))

Loading chunks:   5%|▍         | 16/332 [00:03<01:02,  5.05it/s]/tmp/ipykernel_15995/79809328.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks:  15%|█▌        | 51/332 [00:10<00:58,  4.78it/s]/tmp/ipykernel_15995/79809328.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks:  74%|███████▍  | 245/332 [00:48<00:17,  5.08it/s]/tmp/ipykernel_15995/79809328.py:3: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks: 100%|██████████| 332/332 [01:05<00:00,  5.06it/s]


3788556

In [13]:
df_results_a = []
df_results_b = []
df_results_france = []
for file in tqdm(glob.glob('../data/interim/FranceInitial/*.csv'), desc="Loading chunks"):
    df_tmp = pd.read_csv(file)
    df_tmp_france = df_tmp[df_tmp.LANGUAGE == 'fr']
    if len(df_tmp_france) != 0:
        df_tmp_1 = df_tmp_france.groupby(['DOI'])[['COUNTRY_CODE']].count().merge(df_tmp_france[df_tmp_france.COUNTRY_CODE == 'FR'].groupby(['DOI'])[['COUNTRY_CODE']].count(),
                                                                                  right_index = True, left_index = True)
        df_tmp_1['Percentage'] = df_tmp_1['COUNTRY_CODE_y'] / df_tmp_1['COUNTRY_CODE_x']
        df_results_a.append(list(df_tmp_1[df_tmp_1.Percentage < 0.5].index))
        
        df_tmp_1 = df_tmp_france.groupby(['DOI'])[['COUNTRY_CODE']].count().merge(df_tmp_france[df_tmp_france.COUNTRY_CODE.isna()].groupby(['DOI'])[['COUNTRY_CODE']].count(),
                                                                                  right_index = True, left_index = True)
        df_tmp_1['Percentage'] = df_tmp_1['COUNTRY_CODE_y'] / df_tmp_1['COUNTRY_CODE_x']
        df_results_b.append(list(df_tmp_1[df_tmp_1.Percentage > 0.5].index))
        
        df_results_france.append(list(df_tmp_france.DOI.unique()))
df_results_a = set([doi for sublist in df_results_a for doi in sublist])
df_results_b = set([doi for sublist in df_results_b for doi in sublist])
df_results_france = set([doi for sublist in df_results_france for doi in sublist])  

print('Percentage of pubs in french with more than half of the authors with no French affiliations: ', len(df_results_a) / len(df_results_france))
print('Percentage of pubs in french with more than half of the authors with no country information : ', len(df_results_b) / len(df_results_france))

with open("1_a.pkl", "wb") as f:
    pickle.dump(df_results_a, f)
with open("1_b.pkl", "wb") as f:
    pickle.dump(df_results_b, f)


Loading chunks:   5%|▍         | 16/332 [00:03<01:06,  4.75it/s]/tmp/ipykernel_6453/1518044983.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks:  15%|█▌        | 51/332 [00:11<00:56,  4.96it/s]/tmp/ipykernel_6453/1518044983.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks:  74%|███████▍  | 245/332 [00:53<00:19,  4.38it/s]/tmp/ipykernel_6453/1518044983.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
Loading chunks: 100%|██████████| 332/332 [01:12<00:00,  4.61it/s]


Percentage of pubs in french with more than half of the authors with no French affiliations:  0.0005244034274600715
Percentage of pubs in french with more than half of the authors with no country information :  0.0


2. publications where at least one field is_corresponding is True \
(2.a) how many have more than one corresponding author? \
(2.b) how many have all corresponding authors with only French affiliations? \
(2.c) how many have all corresponding authors with at least one French affiliation? 

In [10]:
df_results_a = []
df_results_b = []
df_results_c = []
df_results_corresponding = []
for file in glob.glob('../data/interim/FranceInitial/*.csv'):
    df_tmp = pd.read_csv(file)
    df_tmp_corresponding = df_tmp[df_tmp.is_corresponding == True]
    df_results_corresponding.append(list(df_tmp_corresponding.DOI.unique()))
    
    df_tmp_1 = df_tmp_corresponding.groupby('DOI')[['COUNTRY_CODE']].count()
    df_results_a.append(list(df_tmp_1[df_tmp_1.COUNTRY_CODE > 1].index))
    
    df_tmp_1 = df_tmp_corresponding.groupby(['DOI'])[['COUNTRY_CODE']].count().merge(df_tmp_corresponding[df_tmp_corresponding.COUNTRY_CODE == 'FR'].groupby(['DOI'])[['COUNTRY_CODE']].count(),
                                                                            right_index = True, left_index = True)
    df_tmp_1['Percentage'] = df_tmp_1['COUNTRY_CODE_y'] / df_tmp_1['COUNTRY_CODE_x']
    df_results_b.append(list(df_tmp_1[df_tmp_1.Percentage == 1].index))
    df_results_c.append(list(df_tmp_1[df_tmp_1.Percentage != 0].index))

df_results_a = set([doi for sublist in df_results_a for doi in sublist])
df_results_b = set([doi for sublist in df_results_b for doi in sublist])
df_results_c = set([doi for sublist in df_results_c for doi in sublist])
df_results_corresponding = set([doi for sublist in df_results_corresponding for doi in sublist])

print('Percentage of pubs with corresponding author with more than one corresponding author: ', len(df_results_a) / len(df_results_corresponding))  
print('Percentage of pubs with corresponding author with only french affiliations : ', len(df_results_b) / len(df_results_corresponding))
print('Percentage of pubs with corresponding author with at least one french affiliations : ', len(df_results_c) / len(df_results_corresponding))    

with open("2_a.pkl", "wb") as f:
    pickle.dump(df_results_a, f)
with open("2_b.pkl", "wb") as f:
    pickle.dump(df_results_b, f)
with open("2_c.pkl", "wb") as f:
    pickle.dump(df_results_c, f)

/tmp/ipykernel_15995/697331482.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_15995/697331482.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_15995/697331482.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)


Percentage of pubs with corresponding author with more than one corresponding author:  0.017066625731866437
Percentage of pubs with corresponding author with only french affiliations :  0.8215078584679282
Percentage of pubs with corresponding author with at least one french affiliations :  0.8219357912556895


3. publications with a single author \
(3.a) how many have only French affiliations? \
(3.b) how many have at least one French affiliation? 

In [17]:
df_results_a = []
df_results_b = []
df_results_unique_author = []
for file in glob.glob('../data/interim/FranceInitial/*.csv'):
    df_tmp = pd.read_csv(file)
    df_tmp_1 = df_tmp.groupby('DOI')[['display_name']].count()
    df_unique_author = df_tmp[df_tmp.DOI.isin(df_tmp_1[df_tmp_1.display_name == 1].reset_index().DOI.unique())]
    df_results_unique_author.append(list(df_unique_author.DOI.unique()))
    
    df_tmp_1 = df_unique_author.groupby(['DOI'])[['COUNTRY_CODE']].count().merge(df_unique_author[df_unique_author.COUNTRY_CODE == 'FR'].groupby(['DOI'])[['COUNTRY_CODE']].count(),
                                                                            right_index = True, left_index = True)
    df_tmp_1['Percentage'] = df_tmp_1['COUNTRY_CODE_y'] / df_tmp_1['COUNTRY_CODE_x']  
    df_results_a.append(list(df_tmp_1[df_tmp_1.Percentage == 1].index))  
    df_results_b.append(list(df_tmp_1[df_tmp_1.Percentage != 0].index))

df_results_a = set([doi for sublist in df_results_a for doi in sublist])
df_results_b = set([doi for sublist in df_results_b for doi in sublist])
df_results_unique_author = set([doi for sublist in df_results_unique_author for doi in sublist])

print('Percentage of pubs with a single author with only french affiliations : ', len(df_results_a) / len(df_results_unique_author)) 
print('Percentage of pubs with corresponding author with at least one french affiliations : ', len(df_results_b) / len(df_results_unique_author))

with open("3_a.pkl", "wb") as f:
    pickle.dump(df_results_a, f)
with open("3_b.pkl", "wb") as f:
    pickle.dump(df_results_b, f)

/tmp/ipykernel_6453/1682900019.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_6453/1682900019.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_6453/1682900019.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)


Percentage of pubs with a single author with only french affiliations :  0.9942830409826343
Percentage of pubs with corresponding author with at least one french affiliations :  0.9942830409826343


4. publications with multiple authors with French affiliations but no corresponding \
(4.a) how many with all authors having only French affiliations? \
(4.b) how many with all authors having at least one French affiliations? 

In [18]:
df_results_a = []
df_results_b = []
df_results_multiple_france_nocorresponding = []
for file in glob.glob('../data/interim/FranceInitial/*.csv'):
    df_tmp = pd.read_csv(file)
    df_tmp_1 = df_tmp.groupby('DOI')[['display_name']].count()
    df_multiple_author = df_tmp[df_tmp.DOI.isin(df_tmp_1[df_tmp_1.display_name > 1].reset_index().DOI.unique())]
    df_france_nocorresponding = df_tmp[(df_tmp.COUNTRY_CODE == 'FR') & (df_tmp.is_corresponding == False)]
    df_tmp_1 = df_multiple_author[df_multiple_author.DOI.isin(df_france_nocorresponding.DOI)]
    df_results_multiple_france_nocorresponding.append(list(df_tmp_1.DOI.unique()))
    
    df_tmp_2 = df_tmp_1.groupby(['DOI'])[['COUNTRY_CODE']].count().merge(df_tmp_1[df_tmp_1.COUNTRY_CODE == 'FR'].groupby(['DOI'])[['COUNTRY_CODE']].count(),
                                                                         right_index = True, left_index = True)
    df_tmp_2['Percentage'] = df_tmp_2['COUNTRY_CODE_y'] / df_tmp_2['COUNTRY_CODE_x']  
    df_results_a.append(list(df_tmp_2[df_tmp_2.Percentage == 1].index))  
    df_results_b.append(list(df_tmp_2[df_tmp_2.Percentage != 0].index))  
    
df_results_a = set([doi for sublist in df_results_a for doi in sublist])
df_results_b = set([doi for sublist in df_results_b for doi in sublist])
df_results_multiple_france_nocorresponding = set([doi for sublist in df_results_multiple_france_nocorresponding for doi in sublist])
print('Percentage of pubs with multiple no corresponding authors with french affiliations with only french affiliations : ', len(df_results_a) / len(df_results_multiple_france_nocorresponding))
print('Percentage of pubs with multiple no corresponding authors with at least one french affiliations : ', len(df_results_b) / len(df_results_multiple_france_nocorresponding))

with open("4_a.pkl", "wb") as f:
    pickle.dump(df_results_a, f)
with open("4_b.pkl", "wb") as f:
    pickle.dump(df_results_b, f)

/tmp/ipykernel_6453/3435045496.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_6453/3435045496.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)
/tmp/ipykernel_6453/3435045496.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmp = pd.read_csv(file)


Percentage of pubs with multiple no corresponding authors with french affiliations with only french affiliations :  0.822484791998637
Percentage of pubs with multiple no corresponding authors with at least one french affiliations :  1.0


### Intersection

In [3]:
option_a = '1'
scenario_a = 'a'

option_b = '2'
scenario_b = 'b'

with open(f'{option_a}_{scenario_a}.pkl', 'rb') as f:
    set_a = pickle.load(f)

with open(f'{option_b}_{scenario_b}.pkl', 'rb') as f:
    set_b = pickle.load(f)
    
print(f'We are comparing option {option_a} and scenario {scenario_a} with option {option_b} and {scenario_b}',len(set_a.intersection(set_b)))

We are comparing option 1 and scenario a with option 2 and b 40
